In [50]:
from dyn_net.dynamical_systems import get_drift
from dyn_net.noise import get_noise
from dyn_net.integrator.params import EulerMaruyamaParams
from dyn_net.dynamical_systems.stats_registry import get_stats
from dyn_net.utils.stats import open_stats_writer, close_stats_writer
from dyn_net.utils.state import open_state_writer, close_state_writer
from dyn_net.networks import get_network
from dyn_net.utils.network import update_system_params_for_network
import numpy as np
from dyn_net.dynamical_systems.jit_double_well_network import double_well_network_chunk , build_double_well_network_kernel_params
from dyn_net.dynamical_systems.double_well_network import build_initial_condition
from dyn_net.integrator.jit import integrate_chunked_jit_timed


In [51]:
from dyn_net.utils.criticality import find_theta_c_from_degree_distribution
params_noise = {
"sigma": 0.4,
}

degree_distribution = {
    'name' : "poisson",
    'params' : {
        'lambda' : 5
    }
} 


theta_c = find_theta_c_from_degree_distribution(
    degree_distribution=degree_distribution,
    sigma = params_noise["sigma"],
    theta_bracket=(0,1))

print(theta_c)

0.017416234439947848


In [52]:
# Number of agents
n = 1000
# Dynamical System definition
system = "double_well_network"
params_system = {
    "theta": 1.2*theta_c ,
    "alpha_rot" : 0,
}


# Network definition
network_name = "configuration_model"

degree_distribution = {
    'name' : "poisson",
    'params' : {
        'lambda' : 5
    }
} 

network_params = {
    'n' : n,
    'degree_distribution': degree_distribution
}

# Noise definition
params_noise = {
    "sigma": 0.4,
}

# Parameters integrator
p_int = EulerMaruyamaParams(
    tmin=0.0,
    tmax=5_000.0,
    dt=0.01,
    stats_every=100,
    state_every=1_000,
    write_stats_at_start=True,
    write_state_at_start=True,
)

# Initial condition

x0 = build_initial_condition({"type": "normal","std":1},n=n,rng=np.random.default_rng())


In [53]:
build_net, p_net = get_network(network_name, network_params)
A = build_net(p_net)

update_system_params_for_network(params_system, network_name, network_params, A)

F, pF = get_drift(system, params_system)
G, pG = get_noise("additive_gaussian", params_noise)
stats_fn, stats_fields = get_stats(system)
kernel_params = build_double_well_network_kernel_params(pF, pG)

stats_writer = open_stats_writer(
    "stats.h5",
    fieldnames=stats_fields,
)

state_writer = open_state_writer(
    "state.h5",
    dim=len(x0),
)


In [54]:
try:
    x_final, timings = integrate_chunked_jit_timed(
        double_well_network_chunk,
        x0,
        params_int=p_int,
        kernel_params=kernel_params,
        stats_fn=stats_fn,
        stats_writer=stats_writer,
        stats_params=pF,
        state_writer=state_writer
    )
    print(timings)
finally:
    close_stats_writer(stats_writer)
    close_state_writer(state_writer)


{'loop_s': 26.195047499844804, 'write_stats_s': 2.962368919979781, 'write_state_s': 0.20577512541785836, 'compute_s': 23.026903454447165, 'steps': 500000}
